### Scraping from Pro-Football Reference

#### Eric Au

Example Source link: https://www.pro-football-reference.com/draft/2022-combine.htm

- Below code summarizes the steps for scraping the last 22 years of NFL combine data from Pro-Football Reference (2000-2022). 

In [10]:
# import needed libraries
from urllib.request import urlopen
from bs4 import BeautifulSoup
import numpy as np
import pandas as pd
import requests
 

In [11]:
# use beatifulsoup and establish request from url
url = 'https://www.pro-football-reference.com'
year = 2022

r = requests.get(url + '/draft/' + str(year) + '-combine.htm')
soup = BeautifulSoup(r.content, 'html.parser')
parsed_table = soup.find_all('table')[0] 

In [3]:
# create a scraper function that will iterate through each page within years
def scrape_combine_data(years):
    
    """ Function takes in a list of years and will iterate through the source page url and fill 
    a dataframe with rows on each year page"""
    
    dataframe = pd.DataFrame()
    
    # loop through each year
    for y in years:
        
        # season to scrape
        year = y
        
        # URL to scrape, notice f string:
        url = f"https://www.pro-football-reference.com/draft/{year}-combine.htm"
        
        # collect HTML data
        html = urlopen(url)
        
        # create beautiful soup object from HTML
        soup = BeautifulSoup(html, features="lxml")
        
        # use getText()to extract the headers into a list
        headers = [th.getText() for th in soup.findAll('tr')[0].findAll('th')]
        headers.append('Year')

        
        # next, grab all data from rows 
        rows = soup.findAll('tr')[1:]
        rows_data = [[td.getText() for td in rows[i].findAll('th')] # add player column
                     + [td.getText() for td in rows[i].findAll('td')] # add all other columns
                     + [year] # add year to 'Year' for each row
                        for i in range(len(rows))] 

        
        # create a dataframe with all aquired info
        dataframe = pd.concat([dataframe, pd.DataFrame(rows_data, columns = headers)], axis=0)
        
        # status
        print('Added to dataframe:', year)
        
    return dataframe
        

In [4]:
# set years between 2000-2022, 2000 is earliest profootball ref keeps info on combine
years = np.arange(2000,2023)
years

array([2000, 2001, 2002, 2003, 2004, 2005, 2006, 2007, 2008, 2009, 2010,
       2011, 2012, 2013, 2014, 2015, 2016, 2017, 2018, 2019, 2020, 2021,
       2022])

In [5]:
# apply scraper function and assign to df
df = scrape_combine_data(years)

Added to dataframe: 2000
Added to dataframe: 2001
Added to dataframe: 2002
Added to dataframe: 2003
Added to dataframe: 2004
Added to dataframe: 2005
Added to dataframe: 2006
Added to dataframe: 2007
Added to dataframe: 2008
Added to dataframe: 2009
Added to dataframe: 2010
Added to dataframe: 2011
Added to dataframe: 2012
Added to dataframe: 2013
Added to dataframe: 2014
Added to dataframe: 2015
Added to dataframe: 2016
Added to dataframe: 2017
Added to dataframe: 2018
Added to dataframe: 2019
Added to dataframe: 2020
Added to dataframe: 2021
Added to dataframe: 2022


In [6]:
# reset index
df.reset_index()

,index,Player,Pos,School,College,Ht,Wt,40yd,Vertical,Bench,Broad Jump,3Cone,Shuttle,Drafted (tm/rnd/yr),Year
0,0,John Abraham,OLB,South Carolina,,6-4,252,4.55,,,,,,New York Jets / 1st / 13th pick / 2000,2000
1,1,Shaun Alexander,RB,Alabama,College Stats,6-0,218,4.58,,,,,,Seattle Seahawks / 1st / 19th pick / 2000,2000
2,2,Darnell Alford,OT,Boston Col.,,6-4,334,5.56,25.0,23,94,8.48,4.98,Kansas City Chiefs / 6th / 188th pick / 2000,2000
3,3,Kyle Allamon,TE,Texas Tech,,6-2,253,4.97,29.0,,104,7.29,4.49,,2000
4,4,Rashard Anderson,CB,Jackson State,,6-2,206,4.55,34.0,,123,7.18,4.15,Carolina Panthers / 1st / 23rd pick / 2000,2000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7816,325,Devonte Wyatt,DT,Georgia,College Stats,6-3,304,4.77,29.0,,111,,,Green Bay Packers / 1st / 28th pick / 2022,2022
7817,326,Jalen Wydermyer,TE,Texas A&M,College Stats,6-4,255,,,,,,,,2022
7818,327,Cade York,K,LSU,College Stats,6-1,206,,,12,,,,Cleveland Browns / 4th / 124th pick / 2022,2022
7819,328,Nick Zakelj,OT,Fordham,,6-6,316,5.13,28.5,27,110,7.75,4.71,San Francisco 49ers / 6th / 187th pick / 2022,2022


In [12]:
df.head()

,Player,Pos,School,College,Ht,Wt,40yd,Vertical,Bench,Broad Jump,3Cone,Shuttle,Drafted (tm/rnd/yr),Year
0,John Abraham,OLB,South Carolina,,6-4,252,4.55,,,,,,New York Jets / 1st / 13th pick / 2000,2000
1,Shaun Alexander,RB,Alabama,College Stats,6-0,218,4.58,,,,,,Seattle Seahawks / 1st / 19th pick / 2000,2000
2,Darnell Alford,OT,Boston Col.,,6-4,334,5.56,25.0,23,94,8.48,4.98,Kansas City Chiefs / 6th / 188th pick / 2000,2000
3,Kyle Allamon,TE,Texas Tech,,6-2,253,4.97,29.0,,104,7.29,4.49,,2000
4,Rashard Anderson,CB,Jackson State,,6-2,206,4.55,34.0,,123,7.18,4.15,Carolina Panthers / 1st / 23rd pick / 2000,2000


In [8]:
df = df.reset_index().drop(columns = 'index')

In [9]:
df

,Player,Pos,School,College,Ht,Wt,40yd,Vertical,Bench,Broad Jump,3Cone,Shuttle,Drafted (tm/rnd/yr),Year
0,John Abraham,OLB,South Carolina,,6-4,252,4.55,,,,,,New York Jets / 1st / 13th pick / 2000,2000
1,Shaun Alexander,RB,Alabama,College Stats,6-0,218,4.58,,,,,,Seattle Seahawks / 1st / 19th pick / 2000,2000
2,Darnell Alford,OT,Boston Col.,,6-4,334,5.56,25.0,23,94,8.48,4.98,Kansas City Chiefs / 6th / 188th pick / 2000,2000
3,Kyle Allamon,TE,Texas Tech,,6-2,253,4.97,29.0,,104,7.29,4.49,,2000
4,Rashard Anderson,CB,Jackson State,,6-2,206,4.55,34.0,,123,7.18,4.15,Carolina Panthers / 1st / 23rd pick / 2000,2000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7816,Devonte Wyatt,DT,Georgia,College Stats,6-3,304,4.77,29.0,,111,,,Green Bay Packers / 1st / 28th pick / 2022,2022
7817,Jalen Wydermyer,TE,Texas A&M,College Stats,6-4,255,,,,,,,,2022
7818,Cade York,K,LSU,College Stats,6-1,206,,,12,,,,Cleveland Browns / 4th / 124th pick / 2022,2022
7819,Nick Zakelj,OT,Fordham,,6-6,316,5.13,28.5,27,110,7.75,4.71,San Francisco 49ers / 6th / 187th pick / 2022,2022


In [14]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7821 entries, 0 to 7820
Data columns (total 14 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   Player               7821 non-null   object
 1   Pos                  7821 non-null   object
 2   School               7821 non-null   object
 3   College              7821 non-null   object
 4   Ht                   7821 non-null   object
 5   Wt                   7821 non-null   object
 6   40yd                 7821 non-null   object
 7   Vertical             7821 non-null   object
 8   Bench                7821 non-null   object
 9   Broad Jump           7821 non-null   object
 10  3Cone                7821 non-null   object
 11  Shuttle              7821 non-null   object
 12  Drafted (tm/rnd/yr)  7821 non-null   object
 13  Year                 7821 non-null   int64 
dtypes: int64(1), object(13)
memory usage: 855.5+ KB


In [20]:
df['Wt'].value_counts()

Wt     141
211    101
197    100
205    100
204     98
      ... 
349      1
163      1
359      1
375      1
356      1
Name: Wt, Length: 210, dtype: int64

In [35]:
repeat_headers = df[df['Wt'] == 'Wt'].index

In [36]:
df.drop(repeat_headers, inplace = True)

In [ ]:
df.to_csv('combine_2000_2022(1).csv')